## Automated Inventory Management System

The inventory management system is a data pipeline that facilitates the tracking of inventory and updates it based on incoming data. The system leverages several software tools and technologies to perform these tasks, including Python, Pandas, and xlwings libraries, and a macro-enabled Excel workbook.

The system consists of several distinct steps. First, the system retrieves inventory data from a macro-enabled Excel workbook and loads it into a Pandas DataFrame. Second, the system retrieves invoice data from another source, such as a separate Excel workbook or a database, and loads it into a separate Pandas DataFrame. Third, the system processes the invoice data to identify which products were sold and updates the inventory data accordingly, using the product ID as a foreign key. Finally, the updated inventory data is loaded back into the same macro-enabled Excel workbook using the xlwings library.

In [11]:
import pandas as pd
import numpy as np

# import the excel file and sheet into a dataframe.
# we will use the 'test_prototype (2)' for testing.

prototype = pd.read_excel("inventory_wb_1.xlsm", sheet_name= "inventory_1")

print("This is how the first 5 records of the Inventory Software look like")
prototype.head()

This is how the first 5 records of the Inventory Software look like


,product_id,Description,Product,Brand,Category,Storage,Packages,Units per Pack,Size,Individual Units,Total Units
0,898,KENS +MAYO XHVY TFF GF,Mayonneise,KENS,Dressing,Dry,2,4,1 GA,2,10
1,3113,VEGALENE PAN COATING VEG BLND AERO,Pan Coating,VEGALENE,Naan,Dry,1,6,14 oz,4,10
2,27182,SMUCKERS JAM STRWBRRY CUP TFF GF,Strawberry Jelly,SMUCKERS,Condiment,Dry,5,200,0.5 oz,100,1100
3,28051,BARILLA PASTA ZITI TFF,Pasta Ziti,BARILLA,Pasta,Dry,4,2,10 LB,1,9
4,28059,BARILLA PASTA PENNE RIGATE TFF,Pasta Penne,BARILLA,Pasta,Dry,7,2,10 lb,1,15


#### Importing the Invoice File into pandas

* Ideally, there is a digital invoice available for PDF download (would make the whole data extraction easier). However, a simple scan of the physical invoice should be enough. Since I do not have any scanned invoice then I manually inputted the data into an excel file, kind of like imitating how it would look like after extracting the data programatically. There are some python libraries that help with this: openCV & Tesseract.

In [12]:
invoice = pd.read_excel("inventory_wb_1.xlsm", sheet_name="invoice")
print("First 5 invoice records")
invoice.head()

First 5 invoice records


,Item No.,Ordered,Shipped,Pack,Size,Unit,Description,Price,Extension
0,901172,1,0,6,#10 CN,NaN,LIBBYS V? BEET MED SLCD,41,0
1,863474,1,1,6,#10 CN,NaN,WEST CRK BEAN GARBANZO,32,32
2,863498,1,1,6,#10 CN,NaN,WEST CRK BEET PICKLED SLCD,41,41
3,39125,3,3,6,#10 CN,NaN,ANG MIA SAUCE MARINARA TFF,54,162
4,629869,1,1,6,#10 CN,NaN,CONTIGO +BEAN BLK,33,33


#### Methodology

* The whole idea is to use the primary key (Item No/product_id) from the invoice, record the value of N Shipped record, and add it to the Inventory Main Prototype Software/Sheet.

In [13]:
for index, row in invoice.iterrows():
    product_id = row['Item No.']
    shipped = row['Shipped']
    prototype.loc[prototype['product_id'] == product_id, 'Packages'] += shipped

In [7]:
prototype.head()

,product_id,Description,Product,Brand,Category,Storage,Packages,Units per Pack,Size,Individual Units,Total Units
0,898,KENS +MAYO XHVY TFF GF,Mayonneise,KENS,Dressing,Dry,3,4,1 GA,2,10
1,3113,VEGALENE PAN COATING VEG BLND AERO,Pan Coating,VEGALENE,Naan,Dry,2,6,14 oz,4,10
2,27182,SMUCKERS JAM STRWBRRY CUP TFF GF,Strawberry Jelly,SMUCKERS,Condiment,Dry,10,200,0.5 oz,100,1100
3,28051,BARILLA PASTA ZITI TFF,Pasta Ziti,BARILLA,Pasta,Dry,6,2,10 LB,1,9
4,28059,BARILLA PASTA PENNE RIGATE TFF,Pasta Penne,BARILLA,Pasta,Dry,11,2,10 lb,1,15


## Exporting the new data into the Excel file

In [14]:
import pandas as pd
import xlwings as xw


# Create a new sheet with the data
wb = xw.Book("inventory_wb_1.xlsm")
ws = wb.sheets.add("Inventory_Updated")
ws.range("A1").options(index=False).value = prototype

# Save the workbook
wb.save()

In [10]:
prototype = pd.read_excel("inventory_wb.xlsm", sheet_name= "Inventory_Updated")
prototype.head()

,Unnamed: 0,product_id,Description,Product,Brand,Category,Storage,Packages,Units per Pack,Size,Individual Units,Total Units
0,0,898,KENS +MAYO XHVY TFF GF,Mayonneise,KENS,Dressing,Dry,3,4,1 GA,2,10
1,1,3113,VEGALENE PAN COATING VEG BLND AERO,Pan Coating,VEGALENE,Naan,Dry,2,6,14 oz,4,10
2,2,27182,SMUCKERS JAM STRWBRRY CUP TFF GF,Strawberry Jelly,SMUCKERS,Condiment,Dry,10,200,0.5 oz,100,1100
3,3,28051,BARILLA PASTA ZITI TFF,Pasta Ziti,BARILLA,Pasta,Dry,6,2,10 LB,1,9
4,4,28059,BARILLA PASTA PENNE RIGATE TFF,Pasta Penne,BARILLA,Pasta,Dry,11,2,10 lb,1,15


## Repeat the Process for continous Automate Update

In [ ]:
# from now on, the previously created sheet will be of reference to update the inventory, that's why there is another sheet_name in the code
# load the sheet into the prototype variable
prototype = pd.read_excel("inventory_wb_1.xlsm", sheet_name= "Inventory_Updated")

# load the new invoice
# invoice = pd.read_excel("inventory_wb_1.xlsm", sheet_name="invoice") # load the new invoice

# update the values based on the invoice
for index, row in invoice.iterrows():
    product_id = row['Item No.']
    shipped = row['Shipped']
    prototype.loc[prototype['product_id'] == product_id, 'Packages'] += shipped

# export it into the same excel sheet for update

'''
To replace an existing sheet with the new data, we use the xw.sheets property to 
reference the existing sheet, and then overwrite its contents with the new data.

'''

import pandas as pd
import xlwings as xw

# Open the workbook and the existing sheet
wb = xw.Book("inventory_wb_1.xlsm")
ws = wb.sheets["Inventory_Updated"]

# Overwrite the existing data with the new data
ws.clear_contents()
ws.range("A1").options(index=False).value = prototype

# Save the changes to the workbook
wb.save()
